In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")

In [3]:
import ast
import json
import numpy as np
import pandas as pd

from util import bfs, count_edge_length, count_edge_crossings

# In context learning prompts

In this notebook we will create a set of prompts to test the in context learning capabilities of ChatGPT when applied to graph visualization problems.

In [4]:
seed = 42
np.random.seed(seed)

## Rank assignment prompts

We will start with the problem of rank assignment. We will generate the new prompts by randomly sampling 5 graphs from different files, and prepending their correct solutions to the prompt asking for the solution of the current graph.

In [5]:
prompt_base = """
Perform a rank assignment on the graph. Use node 0 as a source for the graph. Each node must be assigned to a rank that is equal to the shortest path between that node and the source. Thus, node 0 will be assigned to rank 0, and the neighbors of node 0 will be assigned to rank 1. 
Write no explanations, only respond with the id of each node and the rank it has been assigned to in a format <id> - <rank>.\n
"""

In [6]:
rank_prompt_query_dir = "queries/rank_prompts"
all_rank_prompt_files = set(os.listdir(rank_prompt_query_dir))

In [7]:
def read_prompt_edge_list(query: str) -> list:
    rank_prompt_edge_list = (
        query.split("edge connections:")[1]
        .split("Perform a rank assignment")[0]
        .strip()
    )
    rank_prompt_edge_list = list(ast.literal_eval(rank_prompt_edge_list))
    return rank_prompt_edge_list

def rank_assignment_to_formatted_str(rank_assignment: dict) -> str:
    rank_assignment_str = ""
    for rank, nodes in rank_assignment.items():
        for node in nodes:
            rank_assignment_str += f"{node} - {rank}\n"
    return rank_assignment_str

def minimize_prompt(query: str) -> str:
    query = query.split("\n")
    query = query[:-2]
    query = "\n".join(query)
    return query

In [8]:
k_samples = 3
sample_size = 100

# Sample `sample_size` queries from the all_rank_prompt_files
sampled_rank_prompt_files = np.random.choice(
    list(all_rank_prompt_files), sample_size, replace=False
)
print(f"Sampled {len(sampled_rank_prompt_files)} queries")

Sampled 100 queries


In [9]:
rank_prompts_icl = {}

for rank_prompt_file in sampled_rank_prompt_files:
    # Sample 5 other prompt files different from the current one
    other_rank_prompt_files = np.random.choice(
        list(all_rank_prompt_files - {rank_prompt_file}), k_samples, replace=False
    )
    assert rank_prompt_file not in other_rank_prompt_files

    # Read the current prompt and the other prompts
    rank_prompt = open(os.path.join(rank_prompt_query_dir, rank_prompt_file)).read()
    other_rank_prompts = [
        open(os.path.join(rank_prompt_query_dir, other_rank_prompt_file)).read()
        for other_rank_prompt_file in other_rank_prompt_files
    ]

    # Extract the edge list for both the current prompt and the other prompts
    rank_prompt_edge_list = read_prompt_edge_list(rank_prompt)
    other_rank_prompt_edge_lists = [
        read_prompt_edge_list(other_rank_prompt)
        for other_rank_prompt in other_rank_prompts
    ]

    # Compute the correct rank assignment for the other prompts
    other_rank_prompt_rank_assignments = [
        bfs(other_rank_prompt_edge_list, 0)
        for other_rank_prompt_edge_list in other_rank_prompt_edge_lists
    ]

    # Convert the rank assignments in the expected format, i.e. <id> - <rank> one per row
    other_rank_prompt_rank_assignments_str = [
        rank_assignment_to_formatted_str(other_rank_prompt_rank_assignment)
        for other_rank_prompt_rank_assignment in other_rank_prompt_rank_assignments
    ]

    # Build the prompt
    prompt = [
        "Input:\n{}\nAnswer:\n{}\n".format(
            minimize_prompt(other_rank_prompts[i]),
            other_rank_prompt_rank_assignments_str[i],
        )
        for i in range(k_samples)
    ]
    prompt = prompt_base + "".join(prompt)
    prompt += "Input:\n{}\nAnswer:\n".format(minimize_prompt(rank_prompt))

    # expected_answer = bfs(rank_prompt_edge_list, 0)
    # expected_answer_str = rank_assignment_to_formatted_str(expected_answer)

    rank_prompts_icl[rank_prompt_file] = prompt.strip()

    del rank_prompt, other_rank_prompts, prompt


In [10]:
rank_prompts_icl_query_dir = "queries/rank_prompts_icl"
os.makedirs(rank_prompts_icl_query_dir, exist_ok=True)

In [11]:
# Write the prompts to disk
for rank_prompt_file, prompt in rank_prompts_icl.items():
    with open(os.path.join(rank_prompts_icl_query_dir, rank_prompt_file), "w") as f:
        f.write(prompt)

## Transpose prompts

We can repeat a similar process for the crossing minimization prompts.

In [5]:
prompt_base = """We want to reduce crossings on a graph drawing.
We want to order the nodes in the layers so that there are few crossings in the graph. 
- visit every rank once, starting from layer 0
- try different transpositions of the nodes in that layer
- count the crossings for every transposition. There is a crossing between two edges e1 and e2 if the source of e1 comes before the source of e2, and the target of e1 comes after the target of e2 in the order of nodes in a layer.
- record the transposition that produces the least amount of crossings, and sort the nodes accordingly.
Nodes can NOT be moved to a different layer. You can only reorder nodes within layers.
Write no code and no explanation.
Return the layers dictionary with the nodes ordered, in a code block. I want it formatted like this: {<layer_id>:[<list of ordered nodes>]}"""

In [6]:
transpose_query_dir = "queries/transpose_prompts3"
all_transpose_files = set(os.listdir(transpose_query_dir))

In [7]:
k_samples = 3
sample_size = 100

# Sample `sample_size` queries from the all_rank_prompt_files
sampled_transpose_files = np.random.choice(
    list(all_transpose_files), sample_size, replace=False
)
print(f"Sampled {len(sampled_transpose_files)} queries")

Sampled 100 queries


In [8]:
transpose_queries = {}

for query_file in sorted(os.listdir(transpose_query_dir)):
    transpose_queries[query_file] = {}
    query_file_path = os.path.join(transpose_query_dir, query_file)

    query_str = open(query_file_path, "r").read().strip()

    query_edges = query_str.split("edges = ")[1]
    query_edges = query_edges.split("\n")[0].strip()
    query_edges = ast.literal_eval(query_edges)

    query_ranks = query_str.split("ranks = ")[1]
    query_ranks = query_ranks.split("\n\n")[0].strip()
    query_ranks = query_ranks.split("\n")
    # From each substring remove "Layer " at the 
    # beginning and add "," at the end
    query_ranks = [r[6:].strip() + "," for r in query_ranks]
    query_ranks = "".join(query_ranks)
    query_ranks = "{" + query_ranks[:-1] + "}"
    query_ranks = ast.literal_eval(query_ranks)
    
    transpose_queries[query_file]["edges"] = query_edges
    transpose_queries[query_file]["ranks"] = query_ranks

print("Number of queries: ", len(transpose_queries))

Number of queries:  134


In [9]:
def format_query_to_str(edges: list, ranks: dict) -> str:
    """Format a query to a string"""
    query = []
    query.append("This is the list of edges. Every edge has [<source_id>, <target_id>]:")
    query.append("edges = {}".format(edges))
    query.append("This is the description of what nodes are contained in what layer: ")
    for layer, nodes in ranks.items():
        if layer == 0:
            query.append("ranks = Layer {}: {}".format(layer, nodes))
        else:
            query.append("Layer {}: {}".format(layer, nodes))
    query = "\n".join(query)
    return query

def format_ground_truth(ranks: dict) -> str:
    """Format the ground truth to a string"""
    gt = {int(k): v for k, v in ranks.items()}
    
    ground_truth = "{\n"
    for layer, nodes in gt.items():
        ground_truth += f"{layer}: {nodes},\n"
    ground_truth = ground_truth[:-2] + "\n}"
    return ground_truth
    

In [10]:
# Let's load the ground truth answers for the minimal number of crossings for each query
strasifimal_results = json.load(open("stratisfimal_results/optimal_ranks_10_11.json", "r"))
print("Number of queries with ground truth: ", len(strasifimal_results))

Number of queries with ground truth:  207


In [11]:
transpose_icl = {}

for transpose_file in sampled_transpose_files:
    # Sample 5 other prompt files different from the current one
    other_transpose_files = np.random.choice(
        list(all_transpose_files - {transpose_file}), k_samples, replace=False
    )
    assert transpose_file not in other_transpose_files

    cur_edges = transpose_queries[transpose_file]["edges"]
    cur_ranks = transpose_queries[transpose_file]["ranks"]
    other_edges = [
        transpose_queries[other_file]["edges"] for other_file in other_transpose_files
    ]
    other_ranks = [
        transpose_queries[other_file]["ranks"] for other_file in other_transpose_files
    ]

    ground_truth = strasifimal_results[transpose_file.split(".txt")[0]]
    other_ground_truth = [
        strasifimal_results[other_file.split(".txt")[0]]
        for other_file in other_transpose_files
    ]

    cur_query = format_query_to_str(cur_edges, cur_ranks)
    other_queries = [
        format_query_to_str(other_edges[i], other_ranks[i]) for i in range(k_samples)
    ]
    other_ground_truth_str = [format_ground_truth(gt) for gt in other_ground_truth]

    # print("Current query: ", cur_edges, cur_ranks)
    # print("Other queries: ", other_edges, other_ranks)

    # print(format_query_to_str(cur_edges, cur_ranks))
    # print("Ground truth: ", ground_truth)
    # print(format_ground_truth(ground_truth))

    # Build the prompt
    prompt = [
        "\nInput:\n{}\nAnswer:\n\n{}\n".format(
            other_queries[i],
            other_ground_truth_str[i],
        )
        for i in range(k_samples)
    ]
    prompt = prompt_base + "\n" + "".join(prompt)
    prompt += "\nInput:\n{}\n\nAnswer:\n".format(cur_query)

    transpose_icl[transpose_file] = prompt.strip()
    del (
        cur_edges,
        cur_ranks,
        other_edges,
        other_ranks,
        ground_truth,
        other_ground_truth,
        prompt,
    )


In [12]:
# tojs = [i.split(".txt")[0].strip() for i in sorted(list(all_transpose_files))]
# json.dump(tojs, open("transpose_prompts3_files.json", "w"), indent=2)

In [13]:
transpose_prompts_icl_query_dir = "queries/transpose_prompts_icl"
os.makedirs(transpose_prompts_icl_query_dir, exist_ok=True)

In [14]:
# Write the prompts to disk
for query_file, prompt in transpose_icl.items():
    query_file_path = os.path.join(transpose_prompts_icl_query_dir, query_file)
    with open(query_file_path, "w") as f:
        f.write(prompt)

## Long edge length prompts

In [5]:
long_edge_prompt_base = """A layered graph is a graph where each node is contained in a single layer.
A layered graph is described by list of directed edges, formatted as [(<source_id>, <target_id>)], and a mapping of layers to the nodes contained within.
Every array in the mapping is a different layer, and the numbers in every array indicate the nodes in that layer.

Count the total edge length. 
The edge length of each edge e is always equal to the absolute value of the number of the layer the target is contained in, minus the number of the layer the source is contained in.
The edge length can't be negative.

Write no explanations and no code. Return the total sum of the lengths.
"""

In [6]:
long_edge_query_dir = "queries/longer_edge_length_prompts_steps"
all_long_edge_files = set(os.listdir(long_edge_query_dir))
print("Number of queries: ", len(all_long_edge_files))

k_samples = 5

Number of queries:  50


In [7]:
def read_long_edge_query(fname:str):
    with open(fname, "r") as query_file:
        query_content = query_file.read().split("\n")

    el = [(int(u), int(v)) for (u, v) in ast.literal_eval(query_content[4].split(":")[1].strip())]

    try:
        nr = {
            0 : [int(x) for x in ast.literal_eval(query_content[8].split(":")[1].strip())],
            1 : [int(x) for x in ast.literal_eval(query_content[9].split(":")[1].strip())],
            2 : [int(x) for x in ast.literal_eval(query_content[10].split(":")[1].strip())],
            3 : [int(x) for x in ast.literal_eval(query_content[11].split(":")[1].strip())],
            4 : [int(x) for x in ast.literal_eval(query_content[12].split(":")[1].strip())],
            5 : [int(x) for x in ast.literal_eval(query_content[13].split(":")[1].strip())],
        }
    except:
        nr = {
            0 : [int(x) for x in ast.literal_eval(query_content[8].split(":")[1].strip())],
            1 : [int(x) for x in ast.literal_eval(query_content[9].split(":")[1].strip())],
            2 : [int(x) for x in ast.literal_eval(query_content[10].split(":")[1].strip())],
            3 : [int(x) for x in ast.literal_eval(query_content[11].split(":")[1].strip())],
            4 : [int(x) for x in ast.literal_eval(query_content[12].split(":")[1].strip())],
        }

    return el, nr

def format_query_to_str_long_edge(edges: list, ranks: dict) -> str:
    """Format a query to a string"""
    query = []
    query.append("List of edges formatted as [<source_id>, <target_id>]:")
    query.append("edges = {}".format(edges))
    query.append("Mapping of layers to nodes:")
    for layer, nodes in ranks.items():
        if layer == 0:
            query.append("ranks = Layer {}: {}".format(layer, nodes))
        else:
            query.append("Layer {}: {}".format(layer, nodes))
    query = "\n".join(query)
    return query

In [8]:
long_edge_queries = {}
long_edge_gt = {}
for query_file in all_long_edge_files:
    query_file_path = os.path.join(long_edge_query_dir, query_file)
    edges, ranks = read_long_edge_query(query_file_path)
    long_edge_queries[query_file] = {"edges": edges, "ranks": ranks}
    gt = count_edge_length(edges, ranks)
    long_edge_gt[query_file] = gt

In [9]:
long_edge_icl = {}
for query_file in all_long_edge_files:
    long_edge_query = long_edge_queries[query_file]
    cur_edges = long_edge_query["edges"]
    cur_ranks = long_edge_query["ranks"]
    cur_gt = long_edge_gt[query_file]

    other_long_edge_files = np.random.choice(
        list(all_long_edge_files - {query_file}), k_samples, replace=False
    )
    assert query_file not in other_long_edge_files

    other_queries = [
        long_edge_queries[other_file] for other_file in other_long_edge_files
    ]
    other_edges = [other_query["edges"] for other_query in other_queries]
    other_ranks = [other_query["ranks"] for other_query in other_queries]
    other_gt = [long_edge_gt[other_file] for other_file in other_long_edge_files]

    # Build the prompt
    prompt = [
        "\n## Input:\n{}\n## Answer:\n{}\n".format(
            format_query_to_str_long_edge(other_edges[i], other_ranks[i]),
            other_gt[i],
        )
        for i in range(k_samples)
    ]
    prompt = long_edge_prompt_base + "".join(prompt)
    prompt += "\n## Input:\n{}\n## Answer:\n".format(
        format_query_to_str_long_edge(cur_edges, cur_ranks)
    )

    long_edge_icl[query_file] = prompt

In [10]:
long_edge_icl_query_dir = "queries/longer_edge_length_prompts_icl"
os.makedirs(long_edge_icl_query_dir, exist_ok=True)

In [11]:
# Write the prompts to disk
for query_file, prompt in long_edge_icl.items():
    query_file_path = os.path.join(long_edge_icl_query_dir, query_file)
    with open(query_file_path, "w") as f:
        f.write(prompt)

## Crossing prompts

In [5]:
crossing_base_prompt = """You are a very advanced program that can help me with counting edge crossings in a bipartite graph.
I will provide you with the arrays of nodes of layers A and B and a list of edges as tuples.
For each edge the first element comes from array A, and the second element comes from array B.

Assuming this is a bipartite graph, count the edge crossings. 
Two edges that share a source or a target can not cross. 
Two edges cross if the order of their sources is opposite to the order of their targets.

Exclude all crossings where edges have the same source or the same target.

Write no explanations and no code. Return the number of edges that cross. There might be no edge crossings - in that case, return 0.
"""

In [6]:
def parse_crossing_query(query: str):
    query = query.split("\n")
    layer_a_str = query[3]
    layer_b_str = query[4]
    edges_str = query[7]

    layer_a = ast.literal_eval(layer_a_str.split("A:")[1].strip())
    layer_b = ast.literal_eval(layer_b_str.split("B:")[1].strip())
    edges = ast.literal_eval(edges_str.split("Tuples:")[1].strip())

    return edges, {0: layer_a, 1: layer_b}

In [7]:
cross_query_dir = "queries/crossings_prompts_steps"
cross_query_files = set(sorted(os.listdir(cross_query_dir)))
print("Number of crossing queries: {}".format(len(cross_query_files)))

k_samples = 5

Number of crossing queries: 94


In [8]:
cross_queries = {}
cross_gt = {}
for query_file in cross_query_files:
    query_str = open(os.path.join(cross_query_dir, query_file)).read()
    edges, ranks = parse_crossing_query(query_str)
    gt = count_edge_crossings(edges, ranks)
    cross_queries[query_file] = {"edges": edges, "ranks": ranks}
    cross_gt[query_file] = int(gt)
    

In [9]:
def format_cross_query(ranks:dict, edges:list):
    query = "Layer arrays:\n"
    query += "A: {}\n".format(ranks[0])
    query += "B: {}\n".format(ranks[1])
    query += "Edge tuples:\n"
    query += "Tuples: {}\n".format(edges)
    return query  


In [10]:
cross_icl = {}
for query_file in cross_query_files:
    cross_query = cross_queries[query_file]
    cur_edges = cross_query["edges"]
    cur_ranks = cross_query["ranks"]
    cur_gt = cross_gt[query_file]

    other_cross_files = np.random.choice(
        list(cross_query_files - {query_file}), k_samples, replace=False
    )
    assert query_file not in other_cross_files

    other_queries = [
        cross_queries[other_file] for other_file in other_cross_files
    ]
    other_edges = [other_query["edges"] for other_query in other_queries]
    other_ranks = [other_query["ranks"] for other_query in other_queries]
    other_gt = [cross_gt[other_file] for other_file in other_cross_files]

    query_str = format_cross_query(cur_ranks, cur_edges)

    # Build the prompt
    prompt = [
        "\n## Input:\n{}\n## Answer:\n{}\n".format(
            format_cross_query(other_ranks[i], other_edges[i]),
            other_gt[i],
        )
        for i in range(k_samples)
    ]
    prompt = crossing_base_prompt + "".join(prompt)
    prompt += "\n## Input:\n{}\n## Answer:\n".format(query_str)

    cross_icl[query_file] = prompt

In [11]:
cross_icl_dir = "queries/crossings_prompts_icl"
os.makedirs(cross_icl_dir, exist_ok=True)

In [12]:
# Write the prompts to disk
for query_file, prompt in cross_icl.items():
    query_file_path = os.path.join(cross_icl_dir, query_file)
    with open(query_file_path, "w") as f:
        f.write(prompt)